Source: https://www.kaggle.com/volodymyrgavrysh/bank-marketing-campaigns-dataset

# Bank marketing campaigns dataset | Opening Deposit

It is a dataset that describing Portugal bank marketing campaigns results.
Conducted campaigns were based mostly on direct phone calls, offering bank client to place a term deposit.
If after all marking afforts client had agreed to place deposit - target variable marked 'yes', otherwise 'no'

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder

In [3]:
df = pd.read_csv("../data/raw/bank-additional-full.csv", delimiter=";")

### 1. Preparation
1.1 Define categories

In [51]:
categories = ["job", "marital", "education", "default", "housing", "loan", "contact", "poutcome", "y"]
df[categories] = df[categories].apply(lambda x: x.astype("category"))

### 2. Duplicates

In [52]:
duplicated = df[df.duplicated()].index
duplicated

Int64Index([ 1266, 12261, 14234, 16956, 18465, 20216, 20534, 25217, 28477,
            32516, 36951, 38281],
           dtype='int64')

In [53]:
df[df.duplicated()]

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
1266,39,blue-collar,married,basic.6y,no,no,no,telephone,may,thu,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.855,5191.0,no
12261,36,retired,married,unknown,no,no,no,telephone,jul,thu,...,1,999,0,nonexistent,1.4,93.918,-42.7,4.966,5228.1,no
14234,27,technician,single,professional.course,no,no,no,cellular,jul,mon,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.962,5228.1,no
16956,47,technician,divorced,high.school,no,yes,no,cellular,jul,thu,...,3,999,0,nonexistent,1.4,93.918,-42.7,4.962,5228.1,no
18465,32,technician,single,professional.course,no,yes,no,cellular,jul,thu,...,1,999,0,nonexistent,1.4,93.918,-42.7,4.968,5228.1,no
20216,55,services,married,high.school,unknown,no,no,cellular,aug,mon,...,1,999,0,nonexistent,1.4,93.444,-36.1,4.965,5228.1,no
20534,41,technician,married,professional.course,no,yes,no,cellular,aug,tue,...,1,999,0,nonexistent,1.4,93.444,-36.1,4.966,5228.1,no
25217,39,admin.,married,university.degree,no,no,no,cellular,nov,tue,...,2,999,0,nonexistent,-0.1,93.200,-42.0,4.153,5195.8,no
28477,24,services,single,high.school,no,yes,no,cellular,apr,tue,...,1,999,0,nonexistent,-1.8,93.075,-47.1,1.423,5099.1,no
32516,35,admin.,married,university.degree,no,yes,no,cellular,may,fri,...,4,999,0,nonexistent,-1.8,92.893,-46.2,1.313,5099.1,no


In [54]:
df = df.drop(duplicated)

### 2. Missing data

Fill unknown columns using a SimpleImputer with most frequently used value.

In [55]:
imp = SimpleImputer(missing_values="unknown", strategy="most_frequent")
df[categories] = imp.fit_transform(df[categories])

Map months and days to a number, using categorical and factorize.

In [56]:
month_categories = pd.Categorical(df["month"], categories=["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"], ordered=True)
day_categories = pd.Categorical(df["day_of_week"], categories=["mon", "tue", "wed", "thu", "fri", "sat", "sun"], ordered=True)

In [57]:
labels, unique = pd.factorize(month_categories, sort=True)
df["month"] = labels

labels, unique = pd.factorize(day_categories, sort=True)
df["day_of_week"] = labels

Use one-hot encoding for every other category/columns with non-number values. 

In [58]:
df = pd.get_dummies(df, columns=categories, drop_first=True)

### 3. Drop undeed data

In [59]:
df = df.drop("duration", axis=1)

### 4. Rename weirdly named columns

In [60]:
df.columns

Index(['age', 'month', 'day_of_week', 'campaign', 'pdays', 'previous',
       'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m',
       'nr.employed', 'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'marital_married',
       'marital_single', 'education_basic.6y', 'education_basic.9y',
       'education_high.school', 'education_illiterate',
       'education_professional.course', 'education_university.degree',
       'default_yes', 'housing_yes', 'loan_yes', 'contact_telephone',
       'poutcome_nonexistent', 'poutcome_success', 'y_yes'],
      dtype='object')

In [65]:
df.rename(columns={ "emp.var.rate": "emp_var_rate", "cons.price.idx": "cons_price_idx", "cons.conf.idx": "cons_conf_idx", "education_basic.6y": "education_basic_6y", "education_basic.9y": "education_basicy_9", "nr.employed": "nr_employed" }, inplace=True)

### 4. Save clean data 

In [66]:
df.to_csv("../data/clean/bank-additional-full.csv", index=False)